In [1]:
# import
from langchain.docstore.document import Document
from langchain.document_loaders import TextLoader
from langchain.embeddings.sentence_transformer import SentenceTransformerEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma

In [4]:
from langchain.llms import HuggingFacePipeline
from transformers import AutoTokenizer, AutoModelForCausalLM

model_id="TheBloke/Llama-2-7b-Chat-GPTQ"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id)

In [14]:
from transformers import pipeline

pipeline = pipeline(
    "text-generation",
    model=model, 
    tokenizer=tokenizer, 
    max_length=2048,
    temperature=0.1,
    do_sample=True,
    device=0,  # -1 for CPU
)

tokenizer.pad_token_id = model.config.eos_token_id


llm = HuggingFacePipeline(
    pipeline=pipeline,
)

In [2]:
embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

In [3]:
docsearch = Chroma(persist_directory='data/chroma', embedding_function=embedding_function)

In [15]:
from langchain.chains import RetrievalQA

qa = RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=docsearch.as_retriever(),
    return_source_documents=True
)

In [18]:
query = "How to contribute to CasaOS as a developer?"
result = qa(
    {
        "query": query,
    }
)

if result["result"]:
    print(result["result"])
else:
    print("No summary available.")



You can contribute to CasaOS as a developer by following the guidelines provided in the "Contribute" section of the "Development" chapter. This will help you understand the process and what is expected of you as a developer.



---

Please let me know if you have any questions or need further clarification on any of the above.


In [19]:
for doc in docsearch.similarity_search(query=query):
    print(doc.metadata)

{'source': '../content/contribute/development.md'}
{'source': '../content/contribute.md'}
{'source': '../content/home.md'}
{'source': '../content/contribute.md'}
